In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering

In [3]:
channel1 = 'Scikitlearn'
channel2 = 'Angular'

In [4]:
df_issue = pd.read_csv(channel2+'/'+channel2+'_issue_tags.csv')

In [5]:
df_issue.head()

,Id,asker,time,content,lines,tags,content_stem
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,[],awesom!\n
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"['time', 'timing', 'real-time', 'angular']","sweet, now we can ask angular2 question in rea..."
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[],ye!\n
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"['button', 'io', 'views', 'angular', 'ios', 'v...",we're miss the [view sourc] [improv thi doc] b...
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,['file'],good idea can you file an issu with repo?\n


In [6]:
len(df_issue[df_issue["tags"] != "[]"])

378280